# Movie Reviews

In [7]:
!pip install qiskit -q
!pip install qiskit_machine_learning

import qiskit
import math
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


DEPRECATION: Loading egg at c:\users\jeeth\appdata\local\programs\python\python312\lib\site-packages\gensim-4.3.2-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\jeeth\appdata\local\programs\python\python312\lib\site-packages\smart_open-7.0.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: C:\Users\jeeth\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


DEPRECATION: Loading egg at c:\users\jeeth\appdata\local\programs\python\python312\lib\site-packages\gensim-4.3.2-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\jeeth\appdata\local\programs\python\python312\lib\site-packages\smart_open-7.0.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: C:\Users\jeeth\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#HyperParameters
vocab_size=5000;
max_length=256; # It is should 2^ num_qubits.

## Data

In [ ]:
from tensorflow import keras
imdb= keras.datasets.imdb
(train_data,train_labels), (test_data, test_labels)= imdb.load_data(num_words=vocab_size)
word_index = imdb.get_word_index()
word_index = {k: (v+3) for k,v in word_index.items()}
train_data=[[w for w in i if not w==2]for i in train_data]
test_data=[[w for w in i if not w==2]for i in test_data]
word_index["<PAD>"]=0
word_index["<START>"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3
itos= dict([(value,key) for (key,value) in word_index.items()])
def decode_review(text):
  return " ".join([itos.get(i,"?") for i in text ])

1641221/1641221 [==============================] - 0s 0us/step


In [ ]:
new_data=[(i,train_labels[j]) for j,i in enumerate(train_data) if len(i)<max_length+1]
new_data=new_data + [(i,test_labels[j]) for j,i in enumerate(test_data) if len(i)<max_length+1]
data,labels= zip(*new_data)
#Padding
num_data=200# How many datapoints to consider
data=data[:200]
labels=labels[:200]
#Padding
data= keras.preprocessing.sequence.pad_sequences(data,maxlen=max_length,padding="post" )

# Splitting the data

from sklearn.model_selection import train_test_split
train_data,test_data,train_labels,test_labels= train_test_split(data,labels,train_size=0.72)

## QNN


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_algorithms.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

from qiskit.quantum_info import Statevector
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel

In [ ]:
import pandas as pd
train_data=pd.DataFrame(train_data)
train_labels=pd.DataFrame(train_labels)
test_data=pd.DataFrame(test_data)
test_labels=pd.DataFrame(test_labels)
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(144, 256)
(56, 256)
(144, 1)
(56, 1)


In [ ]:
train_data_with_labels = pd.concat([train_data, train_labels], axis=1)
train_data_with_labels.columns = list(train_data.columns) + ['Cluster']
train_data_with_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,Cluster
0,1,13,296,4,20,11,6,4435,5,13,...,0,0,0,0,0,0,0,0,0,1
1,1,14,22,16,43,530,973,1622,1385,65,...,0,0,0,0,0,0,0,0,0,1
2,1,13,244,6,87,337,7,628,2219,5,...,0,0,0,0,0,0,0,0,0,0
3,1,13,974,13,69,8,702,930,143,14,...,0,0,0,0,0,0,0,0,0,0
4,1,13,244,1713,1681,8,97,134,243,7,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data_with_labels = pd.concat([test_data, test_labels], axis=1)

# Rename the last column to 'Cluster'
test_data_with_labels.columns = list(test_data.columns) + ['Cluster']
test_data_with_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,Cluster
0,1,38,76,47,77,398,44,4,598,114,...,0,0,0,0,0,0,0,0,0,1
1,1,19,6,318,884,3416,2976,1534,15,153,...,0,0,0,0,0,0,0,0,0,1
2,1,637,30,1202,19,13,16,6,4715,11,...,0,0,0,0,0,0,0,0,0,1
3,1,14,20,47,111,439,3445,19,12,15,...,0,0,0,0,0,0,0,0,0,0
4,1,12,9,878,8,14,42,101,85,22,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
combined_data = pd.concat([train_data_with_labels, test_data_with_labels], axis=0)
combined_data.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,Cluster
0,1,13,296,4,20,11,6,4435,5,13,...,0,0,0,0,0,0,0,0,0,1
1,1,14,22,16,43,530,973,1622,1385,65,...,0,0,0,0,0,0,0,0,0,1
2,1,13,244,6,87,337,7,628,2219,5,...,0,0,0,0,0,0,0,0,0,0
3,1,13,974,13,69,8,702,930,143,14,...,0,0,0,0,0,0,0,0,0,0
4,1,13,244,1713,1681,8,97,134,243,7,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
combined_data.shape

(200, 257)

In [ ]:
# @title UUDagger circuit
def U_circuit(qc,theta):
  qc.u(theta[0],0,0,0)
  qc.cu(theta[1],0,0,0,0,1)
  qc.x(0)
  qc.cu(theta[2],0,0,0,0,1)
  qc.x(0)

  qc.mcx([0, 1], 3)
  qc.cu(theta[3],0,0,0,3,2)
  qc.mcx([0, 1], 3)

  qc.x(0)
  qc.mcx([0, 1], 3)
  qc.cu(theta[4],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(0)

  qc.x(1)
  qc.mcx([0, 1], 3)
  qc.cu(theta[5],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(1)

  qc.x(0)
  qc.x(1)
  qc.mcx([0, 1], 3)
  qc.cu(theta[6],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(0)
  qc.x(1)
  return qc

def U_circuit_rev(qc,theta):
  qc.x(0)
  qc.x(1)
  qc.mcx([0, 1], 3)
  qc.cu(-theta[6],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(0)
  qc.x(1)

  qc.x(1)
  qc.mcx([0, 1], 3)
  qc.cu(-theta[5],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(1)

  qc.x(0)
  qc.mcx([0, 1], 3)
  qc.cu(-theta[4],0,0,0,3,2)
  qc.mcx([0, 1], 3)
  qc.x(0)

  qc.mcx([0, 1], 3)
  qc.cu(-theta[3],0,0,0,3,2)
  qc.mcx([0, 1], 3)

  qc.x(0)
  qc.cu(-theta[2],0,0,0,0,1)
  qc.x(0)

  qc.cu(-theta[1],0,0,0,0,1)

  qc.u(-theta[0],0,0,0)

  return qc





In [ ]:
%%capture
%pip install qiskit-aer

In [ ]:
from qiskit_aer import AerSimulator

In [ ]:
# @title QSVM, 2^0+2^1+...+2^{N-1} features
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
#from qiskit import QuantumCircuit, Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

# Load your dataset
# For example, assuming 'your_data.csv' contains your dataset
#df = pd.read_csv('your_data.csv')

N=3
num_data_points=200
test_size=0.2

zeros_string = '0' * N
Num_shots=1024

num_components=0
for i in range(N):
  num_components+=2**i

df_sampled = combined_data.sample(n=num_data_points, random_state=42)

# Extract features and labels
X = df_sampled.drop('Cluster', axis=1)  # Assuming 'Cluster' is the target variable
y = df_sampled['Cluster']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=num_components)
X_pca = pca.fit_transform(X_scaled)

# Create a new DataFrame with the PCA-transformed features
df_pca = pd.DataFrame(data=X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])

num_clusters=2

kmeans = KMeans(n_clusters=num_clusters, random_state=42)
labels = kmeans.fit_predict(df_pca)

centroids = kmeans.cluster_centers_

# Print centroid 0 and centroid 1
centroid_0 = centroids[0]
centroid_1 = centroids[1]

# Add the cluster labels to the DataFrame
df_pca['Cluster'] = labels

A = df_pca.drop('Cluster', axis=1)  # Assuming 'Cluster' is the target variable
B = df_pca['Cluster']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(A, B, test_size=test_size, random_state=42)



# Evaluate the trained model on the test set
predicted_labels = []
for i in range(len(X_test)):
    x = X_test.iloc[i].values
    qc1 = qiskit.QuantumCircuit(N+1,N)
    qc1 = U_circuit(qc1,x)
    qc1 = U_circuit_rev(qc1,centroid_0)
    qc1.measure([0,1,2],[0,1,2])
    backend = AerSimulator(method='automatic')
    result1 = backend.run(qc1).result()
    p0_qc1 = result1.get_counts(0).get(zeros_string, 0) / Num_shots
    qc2 = qiskit.QuantumCircuit(N+1,N)
    qc2 = U_circuit(qc2,x)
    qc2 = U_circuit_rev(qc2,centroid_1)
    qc2.measure([0,1,2],[0,1,2])
    #backend = AerSimulator(method='automatic')
    result2 = backend.run(qc2).result()
    p0_qc2 = result2.get_counts(0).get(zeros_string, 0) / Num_shots
    #counts_qc2 = execute(qc2, Aer.get_backend('qasm_simulator'), shots=Num_shots).result().get_counts()
    #print("counts-2",counts_qc2)
    #p0_qc2 = counts_qc2.get(zeros_string, 0) / Num_shots
    if p0_qc1>p0_qc2:
      predicted_labels.append(0)
    else:
      predicted_labels.append(1)
    #predicted_label = max(counts, key=counts.get)

#print(actual_labels)
#print(predicted_labels)
#print(y_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

# Display the results
print("Accuracy:", round(accuracy,3))
print("Precision:", round(precision,3))
print("F1 Score:", round(f1,3))
print("Recall:", round(recall,3))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Accuracy: 0.95
Precision: 0.857
F1 Score: 0.923
Recall: 1.0
